In [1]:
import torch
import sys
from pathlib import Path
import pandas as pd
import os
import numpy as np
from PIL import Image

wheels_path = "/kaggle/input/clip-interrogator-wheels-x"
clip_interrogator_whl_path = f"{wheels_path}/clip_interrogator-0.4.3-py3-none-any.whl"
!pip install --no-index --find-links $wheels_path $clip_interrogator_whl_path -q

In [2]:
!pip install --no-index --find-links $wheels_path $clip_interrogator_whl_path -q

In [3]:
!pip install torch clip

ERROR: Could not find a version that satisfies the requirement clip (from versions: none)
ERROR: No matching distribution found for clip


In [4]:
import torch
import sys
from pathlib import Path
import pandas as pd
import os
import numpy as np
from PIL import Image
def make_batches(l, batch_size=16):
    for i in range(0, len(l), batch_size):
        yield l[i:i + batch_size]

path = Path('../input/stable-diffusion-image-to-prompts/')
# sample_submission = pd.read_csv(path / 'sample_submission.csv', index_col='imgId_eId')
images = os.listdir(path/"images")
# image_ids = [i.split('.')[0] for i in images]

device = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
!pip list | grep transformers

transformers                           4.15.0


In [6]:
class CFG:
    device = "cuda"
    seed = 42
    embedding_length = 384
    sentence_model_path = "/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2"
    blip_model_path = "/kaggle/input/clip-interrogator-models-x/model_large_caption.pth"
    ci_clip_model_name = "ViT-H-14/laion2b_s32b_b79k"
    clip_model_name = "ViT-H-14"
    clip_model_path = "/kaggle/input/clip-interrogator-models-x/CLIP-ViT-H-14-laion2B-s32B-b79K/open_clip_pytorch_model.bin"
    cache_path = "/kaggle/input/clip-interrogator-models-x"


In [7]:
import open_clip
import inspect
import importlib

from blip.models import blip
from clip_interrogator import clip_interrogator
from PIL import Image
import torchvision.transforms as transforms



In [8]:
sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer, models

st_model = SentenceTransformer(CFG.sentence_model_path)

In [9]:
# replace tokenizer path to prevent downloading
# blip 모델 모듈 대체
blip_path = inspect.getfile(blip)

fin = open(blip_path, "rt")
data = fin.read()
data = data.replace(
    "BertTokenizer.from_pretrained('bert-base-uncased')", 
    "BertTokenizer.from_pretrained('/kaggle/input/clip-interrogator-models-x/bert-base-uncased')"
)
fin.close()

fin = open(blip_path, "wt")
fin.write(data)
fin.close()

# reload module
importlib.reload(blip)

<module 'blip.models.blip' from '/opt/conda/lib/python3.7/site-packages/blip/models/blip.py'>

In [10]:
# fix clip_interrogator bug
clip_interrogator_path = inspect.getfile(clip_interrogator.Interrogator)

fin = open(clip_interrogator_path, "rt")
data = fin.read()
data = data.replace(
    'open_clip.get_tokenizer(clip_model_name)', 
    'open_clip.get_tokenizer(config.clip_model_name.split("/", 2)[0])'
)
fin.close()

fin = open(clip_interrogator_path, "wt")
fin.write(data)
fin.close()

# reload module
importlib.reload(clip_interrogator)

<module 'clip_interrogator.clip_interrogator' from '/opt/conda/lib/python3.7/site-packages/clip_interrogator/clip_interrogator.py'>

In [11]:
# Step 1: Config 객체 생성
model_config = clip_interrogator.Config(clip_model_name=CFG.ci_clip_model_name)
model_config.cache_path = CFG.cache_path

In [12]:
configs_path = os.path.join(os.path.dirname(os.path.dirname(blip_path)), 'configs')
med_config = os.path.join(configs_path, 'med_config.json')
blip_model = blip.blip_decoder(
    pretrained=CFG.blip_model_path,
    image_size=model_config.blip_image_eval_size, 
    vit=model_config.blip_model_type, 
    med_config=med_config
)
blip_model.eval()
blip_model = blip_model.to(model_config.device)
model_config.blip_model = blip_model

load checkpoint from /kaggle/input/clip-interrogator-models-x/model_large_caption.pth


In [13]:
# CLIP 모델 생성
clip_model = open_clip.create_model(CFG.clip_model_name, precision='fp16' if model_config.device == 'cuda' else 'fp32')

# #학습된 모델 불러오기
open_clip.load_checkpoint(clip_model, CFG.clip_model_path)
clip_model.to(model_config.device).eval()
model_config.clip_model = clip_model

In [14]:
clip_preprocess = transforms.Compose([
     transforms.Resize((224, 224)),
     transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
         mean=[0.5, 0.5, 0.5],
         std=[0.5, 0.5, 0.5]
        #transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711))
     )
 ])
model_config.clip_preprocess = clip_preprocess
# # transforms.Normalize(
# #         mean=[0.5, 0.5, 0.5],
# #         std=[0.229, 0.224, 0.225]

In [15]:
# import open_clip
# clip_preprocess = open_clip.image_transform(
#      clip_model.visual.image_size,
#      is_train = False,
#      mean = getattr(clip_model.visual, 'image_mean', None),
#      std = getattr(clip_model.visual, 'image_std', None),
#  )
# model_config.clip_preprocess = clip_preprocess

In [16]:
# Step 2: Interrogator 객체 생성
ci = clip_interrogator.Interrogator(model_config)

Loaded CLIP model and data in 2.10 seconds.


In [17]:
# Step 3: 프롬프트로 바꿀 이미지 로드
#image = Image.open("/kaggle/input/stable-diffusion-image-to-prompts/images/20057f34d.png")

# **참고**

> https://github.com/pharmapsychotic/clip-interrogator/blob/main/clip_interrogator/clip_interrogator.py#L213

In [18]:
def image_to_features(image: Image) -> torch.Tensor:
        ci._prepare_clip()
        images = ci.clip_preprocess(image).unsqueeze(0).to(ci.device)
        with torch.no_grad(), torch.cuda.amp.autocast():
            image_features = ci.clip_model.encode_image(images)
            image_features /= image_features.norm(dim=-1, keepdim=True)
        return image_features
    


In [19]:
def interrogate_classic(image: Image,max_flavors: int=3) -> str:
        """Classic mode creates a prompt in a standard format first describing the image, 
        then listing the artist, trending, movement, and flavor text modifiers."""
        caption = ci.generate_caption(image)
        image_features = ci.image_to_features(image)

        medium = ci.mediums.rank(image_features, 1)[0]
        movement = ci.movements.rank(image_features, 1)[0]
        flaves = ", ".join(ci.flavors.rank(image_features, max_flavors))

        if caption.startswith(medium):
            prompt = f"{caption} , {movement}, {flaves}"
        else:
            prompt = f"{caption}, {medium} , {movement}, {flaves}"

        return clip_interrogator._truncate_to_fit(prompt, ci.tokenize)




In [20]:
# #이미지에 적용
prompts = []

images_path = "../input/stable-diffusion-image-to-prompts/images/"
for image_name in images:
    img = Image.open(images_path + image_name).convert("RGB")

    generated = interrogate_classic(img)
    
    prompts.append(generated)

100%|██████████| 50/50 [00:00<00:00, 198.74it/s]


In [21]:
# #이미지에 적용
# prompts = []

# images_path = "../input/gustavosta-stable-diffusion-prompts-sd2/eval_images/"
# for image_name in images:
#     img = Image.open(image_name).convert("RGB")

#     generated = interrogate_classic(img)
    
#     prompts.append(generated)

In [22]:
prompts

['a man that is standing in front of a counter, a digital painting , digital art, the mighty donut, digitally painted, donut',
 'a cartoon dinosaur with a piece of cheese in its mouth, an illustration of , digital art, buttercup eating pizza, alosaurus, spinosaurus',
 'a drawing of a robot on a piece of paper, a screenprint , art brut, ((robot)), robot cat, robot design',
 'a circular hole in the middle of a desert, a screenshot , hurufiyya, studying a hell open rift portal, huge black round hole, a still of an ethereal',
 'a man in an astronaut suit walking down a path, a colorized photo , digital art, american astronaut in the forest, astronaut walking, lonely astronaut',
 'a close up of a wooden object on a table, an abstract sculpture , op art, swirling around, carved wood, wood art',
 'a painting of a man with a lizard on his head, a surrealist painting , magic realism, magic realism painting, kent monkman, portrait of circus artist']

In [23]:
df_submission = pd.read_csv(path / 'sample_submission.csv', index_col='imgId_eId')
df_submission.head()

,val
imgId_eId,
20057f34d_0,0.018848
20057f34d_1,0.030190
20057f34d_2,0.072792
20057f34d_3,-0.000673
20057f34d_4,0.016774


In [24]:
imgIds = [i.split('.')[0] for i in images]

eIds = list(range(CFG.embedding_length))

imgId_eId = [
    '_'.join(map(str, i)) for i in zip(
        np.repeat(imgIds, CFG.embedding_length),
        np.tile(range(CFG.embedding_length), len(imgIds))
    )
]

assert sorted(imgId_eId) == sorted(df_submission.index)

In [25]:
#임베딩
embeddings = st_model.encode(prompts).flatten()

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [26]:
embeddings

array([-0.00050107,  0.06069836, -0.06835597, ...,  0.06744548,
       -0.0664781 ,  0.04254934], dtype=float32)

In [27]:
submission = pd.DataFrame(
    index=imgId_eId,
    data=embeddings,
    columns=['val']
).rename_axis('imgId_eId')

In [28]:
submission.head()

,val
imgId_eId,
f27825b2c_0,-0.000501
f27825b2c_1,0.060698
f27825b2c_2,-0.068356
f27825b2c_3,-0.086379
f27825b2c_4,-0.080334


In [29]:
submission.to_csv('submission.csv')

# VIT